In [15]:
import os
import re
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from sklearn.utils import shuffle

def getDataFromFiles(filefolder, category1 = "*", category2= "*", error_del = False, min_len=100, drop_duplicates=False, maxrow_by_category = 0) :
    if category1 == "*" : 
        category1 = set([re.search('_.*_', str(filename)).group().replace("_","") for filename in Path(filefolder).rglob("*.json")])
    else : 
        category1 = category1.split(",")
        
    if category2 == "*" : 
        category2 = set([re.search('_.*\.json', str(filename)).group().replace(".json","").split("_")[2] for filename in Path(filefolder).rglob("*.json")])
    else : 
        category2 = category2.split(",")
    
    df_li = []
    for cate1 in category1 :
        for cate2 in category2 :
            pattern = "*_{}_{}.json".format(cate1, cate2)
            tmp_df_li = []
            for path in Path(filefolder).rglob(pattern):
                try : 
                    tmp_df = pd.read_json(path) 
                    tmp_df = tmp_df[tmp_df['content'].apply(len) > min_len] #지정 길이 이상의 데이터만 조회
                    if drop_duplicates : #중복 뉴스 삭제
                        tmp_df = tmp_df.drop_duplicates(subset='content', keep='first')
                    tmp_df_li.append(tmp_df)
                except Exception as e:
                    if error_del : os.remove(path) #오류가 있는 파일 삭제. 재작업 목적
                    print("Error file : {}".format(path))
                    print("Error message : {}".format(str(e)))
        
            if len(tmp_df_li) > 1 :                
                print("-- Parttern : {} --".format(pattern)) 
                
                # 세분류별 데이터 건수 제한
                tmp_df = pd.concat(tmp_df_li, sort=False)        
                if maxrow_by_category > 0 : tmp_df = shuffle(tmp_df)[:maxrow_by_category]

                df_li.append(tmp_df)    
    df = pd.concat(df_li, sort=False)
    print("{} rows returned".format(df.shape[0]))
    return df

df = getDataFromFiles("../../NaverNews/navernews/news/news/", category1='정치', category2='*', min_len=200, drop_duplicates=True, maxrow_by_category=10000)

Error file : ../../NaverNews/navernews/news/news/2020/03/02/20200302_정치_정치일반.json
Error message : unhashable type: 'list'
Error file : ../../NaverNews/navernews/news/news/2020/03/01/20200301_정치_정치일반.json
Error message : unhashable type: 'list'
-- Parttern : *_정치_정치일반.json --
-- Parttern : *_정치_국회정당.json --
Error file : ../../NaverNews/navernews/news/news/2019/12/17/20191217_정치_청와대.json
Error message : Unexpected character found when decoding array value (2)
-- Parttern : *_정치_청와대.json --
-- Parttern : *_정치_국방외교.json --
-- Parttern : *_정치_북한.json --
-- Parttern : *_정치_행정.json --
60000 rows returned


In [12]:

category2 = set([re.search('_.*\.json', str(filename)).group().replace(".json","").split("_")[2] for filename in Path("../../NaverNews/navernews/news/news/").rglob("*.json")])
category2

{'IT일반',
 '건강정보',
 '게임리뷰',
 '경제일반',
 '공연전시',
 '과학일반',
 '교육',
 '국방외교',
 '국회정당',
 '글로벌경제',
 '금융',
 '날씨',
 '노동',
 '도로교통',
 '모바일',
 '보안해킹',
 '부동산',
 '북한',
 '사건사고',
 '사회일반',
 '산업재계',
 '생활경제',
 '생활문화일반',
 '식품의료',
 '언론',
 '여행레저',
 '음식맛집',
 '인권복지',
 '인물',
 '인터넷SNS',
 '자동차시승기',
 '정치일반',
 '종교',
 '중기벤처',
 '증권',
 '지역',
 '책',
 '청와대',
 '컴퓨터',
 '통신뉴미디어',
 '패션뷰티',
 '행정',
 '환경'}

In [57]:
Path?

Init signature: Path(*args, **kwargs)
Docstring:     
PurePath subclass that can make system calls.

Path represents a filesystem path but unlike PurePath, also offers
methods to do system calls on path objects. Depending on your system,
instantiating a Path will return either a PosixPath or a WindowsPath
object. You can also instantiate a PosixPath or WindowsPath directly,
but cannot instantiate a WindowsPath on a POSIX system or vice versa.
File:           ~/opt/anaconda3/lib/python3.7/pathlib.py
Type:           type
Subclasses:     PosixPath, WindowsPath


In [12]:
df[df['content'].apply(len) > 100]

,url,category1,category2,date,title,media,inputdate,summary,content,oricategory,updatedate,comment,angry,like,sad,warm,want
0,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,행정,2019-02-03,미 국무부 스티브 비건 특별대표 방한,뉴시스,2019.02.03. 오후 5:19,None,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,사회,2020-03-19 09:52:06,NaN,NaN,NaN,NaN,NaN,NaN
1,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,행정,2019-02-03,입국하는 스티븐 비건 특별대표,뉴시스,2019.02.03. 오후 5:19,None,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,사회,2020-03-19 09:52:07,NaN,NaN,NaN,NaN,NaN,NaN
2,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,행정,2019-02-03,고령화하는 이산가족...지난해 4900명 사망,조선일보,2019.02.03. 오후 6:57,None,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,정치,2020-03-19 09:52:07,NaN,NaN,NaN,NaN,NaN,NaN
3,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,행정,2019-02-03,한미 방위비분담금 사실상 타결...10억달러 미만,조선일보,2019.02.03. 오후 8:25,None,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,정치,2020-03-19 09:52:07,NaN,NaN,NaN,NaN,NaN,NaN
4,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,행정,2019-02-03,"韓美, '10억달러 미만·1년'으로 방위비 분담금 이견 좁혀가",노컷뉴스,2019.02.03. 오후 9:48,양대 쟁점이던 총액과 유효기간에 있어 요구사항 맞바꾸는 형식으로 진행,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,정치,2020-03-19 09:52:07,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,북한,2020-01-22,북한 핵실험 중단 약속 얽매일 이유 없어…새로운길 모색(종합2보),연합뉴스,2020.01.22. 오전 2:59,None,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,세계,2020-03-19 09:33:00,NaN,NaN,NaN,NaN,NaN,NaN
220,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,북한,2020-01-22,북한 美 제재 고집하면 새로운 길 모색해야(종합),뉴시스,2020.01.22. 오전 1:05,None,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,세계,2020-03-19 09:33:00,NaN,NaN,NaN,NaN,NaN,NaN
221,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,북한,2020-01-22,[단독]北 “리선권 외무상 임명” 외국공관에 통보,동아일보,2020.01.22. 오전 3:02,None,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,정치,2020-03-19 09:33:00,NaN,NaN,NaN,NaN,NaN,NaN
222,https://news.naver.com/main/read.nhn?mode=LS2D...,정치,북한,2020-01-22,김정은 긴 침묵… 남측 개별관광 제안에 고심 깊은 듯,한국일보,2020.01.22. 오전 4:43,None,// flash 오류를 우회하기 위한 함수 추가\nfunction _flas...,정치,2020-03-19 09:33:02,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
import re
hangul = re.compile('_.*_')
test = "../../NaverNews/navernews/news/news/2019/02/03/20190203_정치_행정.json"

print(re.search(hangul, test).group())

_정치_


In [73]:
file_li = Path("../../NaverNews/navernews/news/news/").rglob("*_정치_*.json") #패턴이 일치하는 파일목록 조회
cate1 = []

for file in file_li :
    cate1t = re.search('_[].*\.json', test).group().replace(".json","")
    if cate1t not in cate1 :
        cate1.append(cate1t)
cate1        

['_정치_행정']

In [61]:
cate1 = []


{'정치'}

In [15]:
df.head()

,url,category1,category2,date,title,media,inputdate,summary,content,oricategory,comment,updatedate,angry,want,like,sad,warm
0,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,경제 일반,2019-12-02,활짝 열릴 ‘오픈뱅킹’ 고객 유혹 총력전,경향신문,2019.12.02. 오후 9:25,None,"ㆍ은행권, 18일 전면 시행 앞두고 각양각색 유인책 잇따라 선보여 ㆍ시범운영 기간...",경제,0,2020-03-17 04:28:59,NaN,NaN,NaN,NaN,NaN
1,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,경제 일반,2019-12-02,한국중재대상 ‘영광의 얼굴들’,한국경제,2019.12.02. 오후 9:57,None,대한상사중재원과 한국경제신문사가 함께 수여하는 ‘제2회 한국중재대상’ 시상식이 2...,경제,0,2020-03-17 04:28:59,NaN,NaN,NaN,NaN,NaN
2,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,경제 일반,2019-12-02,재산세·건보료·車보험료 줄줄이 인상…직장인 '악 소리',TV조선,2019.12.02. 오후 9:33,None,"손해보험사들이 또 자동차 보험료 인상에 시동을 걸었습니다. 지난 1월 3%, 6...",경제,535,2020-03-17 04:29:00,1649.0,5.0,18.0,5.0,7.0
3,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,경제 일반,2019-12-02,러시아-중국 잇는 '시베리아의 힘' 가스관 개통(종합),연합뉴스,2019.12.02. 오후 9:51,None,"푸틴-시진핑, 화상 연결로 개통식 참관…中에 매년 380억㎥씩 30년간 가스 공급 ...",경제,17,2020-03-17 04:29:01,20.0,1.0,8.0,NaN,NaN
4,https://news.naver.com/main/read.nhn?mode=LS2D...,경제,경제 일반,2019-12-02,“건강 위해 정관장 알파프로젝트에 참여하세요”,세계일보,2019.12.02. 오후 10:09,None,2일 서울 중구 충무로 ‘한국의 집’에서 KGC인삼공사 모델들이 현대인에게 8가...,경제,0,2020-03-17 04:29:02,NaN,NaN,NaN,NaN,NaN


In [28]:
df.groupby([df['category1']])['url'].agg(['count'])

,count
category1,
정치,20000


In [32]:
df.groupby([df['category1'], df['category2']])['url'].agg(['count'])

count
category1 category2       
정치        국방/외교        881
          국회/정당       2540
          북한           557
          정치일반        4634
          청와대          744
          행정           644

In [23]:
df[df.category1 == "정치"].to_csv('./data/정치.csv')

In [6]:
df_정치 = df[df.category1 == "정치"]
print(df_정치[df_정치.angry > 10].shape)
print(df_정치[df_정치.want > 10].shape)
print(df_정치[df_정치.like > 10].shape)
print(df_정치[df_정치.sad > 10].shape)
print(df_정치[df_정치.warm > 10].shape)

(5009, 17)
(379, 17)
(3419, 17)
(254, 17)
(424, 17)


In [6]:
cate1 = df.category1.unique()
cate2 = df.category2.unique()
print(cate1)
print(cate2)

['사회' 'IT/과학' '경제' '생활/문화' '정치']
['언론' '보안/해킹' '컴퓨터' '증권' '금융' '생활경제' '환경' '통신/뉴미디어' '부동산' 'IT 일반' '산업/재계'
 '교육' '글로벌 경제' '모바일' '중기/벤처' '게임/리뷰' '경제 일반' '사건사고' '과학 일반' '인터넷/SNS'
 '사회 일반' '지역' '도로/교통' '인물' '건강정보' '북한' '책' '국방/외교' '노동' '종교' '공연/전시'
 '음식/맛집' '국회/정당' '생활문화 일반' '날씨' '행정' '패션/뷰티' '자동차/시승기' '여행/레저' '식품/의료']


In [56]:
import re

cate1 = df.category1.unique()
li = []

for c1 in cate1 :
    if (c1 == "속보") | (c1 == "세계") :
        continue
    print("----- {} ----".format(c1))
    df1 = df[df['category1']==c1]
    cate2 = df1.category2.unique()
    for c2 in cate2:
        print(c2)
        tmp_df = df1[df1['category2']==c2]
        url = tmp_df.iloc[0]['url']
        
        sid1 = int(re.search(r"(sid1=[0-9]*)", url).group().split('=')[1])
        sid2 = int(re.search(r"(sid2=[0-9]*)", url).group().split('=')[1])

        li.append({'category1' : c1
                        ,'category2' : c2
                        ,'filename': "{}_{}".format(c1.replace(" ","").replace("/",""), c2.replace(" ","").replace("/",""))
                        , 'sid1' : sid1
                        , 'sid2' : sid2} )
        
df_cate = pd.DataFrame(li)
df_cate.to_csv('./cate_list.csv')

----- 경제 ----
글로벌 경제 
금융 
산업/재계 
부동산 
중기/벤처 
증권 
경제 일반 
생활경제 
----- IT/과학 ----
과학 일반 
게임/리뷰 
컴퓨터 
통신/뉴미디어 
모바일 
인터넷/SNS 
IT 일반 
보안/해킹 
----- 사회 ----
교육 
노동 
인물 
지역 
인권/복지 
사건사고 
언론 
환경 
사회 일반 
식품/의료 
----- None ----
----- 생활/문화 ----
날씨 
생활문화 일반 
공연/전시 
음식/맛집 
패션/뷰티 
자동차/시승기 
도로/교통 
여행/레저 
종교 
책 
건강정보 
----- 정치 ----
정치일반 
국방/외교 
청와대 
국회/정당 
북한 
행정 


In [57]:
df_cate

,category1,category2,filename,sid1,sid2
0,경제,글로벌 경제,경제_글로벌경제,101,262
1,경제,금융,경제_금융,101,259
2,경제,산업/재계,경제_산업재계,101,261
3,경제,부동산,경제_부동산,101,260
4,경제,중기/벤처,경제_중기벤처,101,771
5,경제,증권,경제_증권,101,258
6,경제,경제 일반,경제_경제일반,101,263
7,경제,생활경제,경제_생활경제,101,310
8,IT/과학,과학 일반,IT과학_과학일반,105,228
9,IT/과학,게임/리뷰,IT과학_게임리뷰,105,229


In [7]:
df_정치.head()

,category1,comment,_type,category2,like,title,url,media,angry,summary,content,warm,updatedate,want,oricategory,date,inputdate,sad
14,정치,1,dict,정치일반,0,"춘천 분구 확정…강원지역 선거구, 원주 빼고 다 바뀐다",https://news.naver.com/main/read.nhn?mode=LS2D...,머니투데이,4,None,김세환 국회의원선거구획정위원회 위원장이 3일 오후 서울 관악구 중앙선거관리위원회 ...,0,2020-03-15 14:10:17,0,정치,2020-03-03,2020.03.03. 오후 6:21,0
17,정치,0,dict,정치일반,0,"홍남기 영남·중부권 감염전문병원 확대, 음압병상도 확충",https://news.naver.com/main/read.nhn?mode=LS2D...,연합뉴스,1,None,관세청 압수 마스크 40만장 풀 것…마스크·방호복 전략물자 지정 검토해야 홍남기 ...,0,2020-03-15 14:10:29,0,정치,2020-03-03,2020.03.03. 오후 6:08,0
18,정치,5240,dict,행정,97,"중국 주민들, 한국 교민 집 각목으로 막고 못질",https://news.naver.com/main/read.nhn?mode=LS2D...,조선일보,14552,None,집에 들어가자 못나오게 만들어 교민의 회사 동료가 각목 제거 중국 안후이성 허페이...,40,2020-03-15 14:10:30,81,정치,2020-03-03,2020.03.03. 오전 1:46,31
21,정치,164,dict,정치일반,34,"이만희, 이번에는 ‘정세균 시계’ 논란..丁 “허무맹랑한 주장”",https://news.naver.com/main/read.nhn?mode=LS2D...,파이낸셜뉴스,264,None,이만희 신천지예수교 증거장막성전 총회장이 2일 기자회견에서 박근혜 전 대통령의 이...,7,2020-03-15 14:10:41,5,정치,2020-03-03,2020.03.03. 오후 5:59,2
22,정치,18,dict,정치일반,63,"[퇴근길] 의사 안철수, 봉사가 '쇼'라고?",https://news.naver.com/main/read.nhn?mode=LS2D...,머니S,15,None,안철수 국민의당 대표 측이 일각에서 제기된 '안 대표는 환자 근처에도 가지 않는다...,0,2020-03-15 14:10:42,2,정치,2020-03-03,2020.03.03. 오후 6:00,2


In [8]:
import numpy as np 

df_정치[df_정치['summary']!=""].head()

,category1,comment,_type,category2,like,title,url,media,angry,summary,content,warm,updatedate,want,oricategory,date,inputdate,sad
14,정치,1,dict,정치일반,0,"춘천 분구 확정…강원지역 선거구, 원주 빼고 다 바뀐다",https://news.naver.com/main/read.nhn?mode=LS2D...,머니투데이,4,None,김세환 국회의원선거구획정위원회 위원장이 3일 오후 서울 관악구 중앙선거관리위원회 ...,0,2020-03-15 14:10:17,0,정치,2020-03-03,2020.03.03. 오후 6:21,0
17,정치,0,dict,정치일반,0,"홍남기 영남·중부권 감염전문병원 확대, 음압병상도 확충",https://news.naver.com/main/read.nhn?mode=LS2D...,연합뉴스,1,None,관세청 압수 마스크 40만장 풀 것…마스크·방호복 전략물자 지정 검토해야 홍남기 ...,0,2020-03-15 14:10:29,0,정치,2020-03-03,2020.03.03. 오후 6:08,0
18,정치,5240,dict,행정,97,"중국 주민들, 한국 교민 집 각목으로 막고 못질",https://news.naver.com/main/read.nhn?mode=LS2D...,조선일보,14552,None,집에 들어가자 못나오게 만들어 교민의 회사 동료가 각목 제거 중국 안후이성 허페이...,40,2020-03-15 14:10:30,81,정치,2020-03-03,2020.03.03. 오전 1:46,31
21,정치,164,dict,정치일반,34,"이만희, 이번에는 ‘정세균 시계’ 논란..丁 “허무맹랑한 주장”",https://news.naver.com/main/read.nhn?mode=LS2D...,파이낸셜뉴스,264,None,이만희 신천지예수교 증거장막성전 총회장이 2일 기자회견에서 박근혜 전 대통령의 이...,7,2020-03-15 14:10:41,5,정치,2020-03-03,2020.03.03. 오후 5:59,2
22,정치,18,dict,정치일반,63,"[퇴근길] 의사 안철수, 봉사가 '쇼'라고?",https://news.naver.com/main/read.nhn?mode=LS2D...,머니S,15,None,안철수 국민의당 대표 측이 일각에서 제기된 '안 대표는 환자 근처에도 가지 않는다...,0,2020-03-15 14:10:42,2,정치,2020-03-03,2020.03.03. 오후 6:00,2


In [1]:
!pip install scrapinghub

In [10]:
df_정치[:1]

,category1,comment,_type,category2,like,title,url,media,angry,summary,content,warm,updatedate,want,oricategory,date,inputdate,sad
14,정치,1,dict,정치일반,0,"춘천 분구 확정…강원지역 선거구, 원주 빼고 다 바뀐다",https://news.naver.com/main/read.nhn?mode=LS2D...,머니투데이,4,None,김세환 국회의원선거구획정위원회 위원장이 3일 오후 서울 관악구 중앙선거관리위원회 ...,0,2020-03-15 14:10:17,0,정치,2020-03-03,2020.03.03. 오후 6:21,0


In [14]:
unicode("▶", "ascci")

NameError: name 'unicode' is not defined

In [60]:
import os
os.makedirs('1/2/3')